In [1]:
import pandas as pd
import sqlite3
import sqlalchemy

In [2]:
conn = sqlite3.connect('ladder.db')

## Views

###### 62) Look at the `yum` table. It is the stock data for Yum! Brands, Inc. from 2015 through 2019. Yum! is the company that owns Taco Bell, the best restaurant.

In [3]:
pd.read_sql("select count(*) from yum;", conn)

,count(*)
0,1257


In [4]:
pd.read_sql("select* from yum limit 5;", conn)

,date,open,high,low,close,volume
0,2015-01-02,52.796551,52.846874,51.761322,52.012939,2283400
1,2015-01-05,52.056076,52.278934,50.948959,50.956146,4418600
2,2015-01-06,51.301224,51.329979,49.877785,50.330696,5004400
3,2015-01-07,51.020847,52.056076,50.884254,51.998562,4554100
4,2015-01-08,52.408340,53.040977,52.365204,52.904385,4258200


###### 63) Query the `yum` table, aggregating by **both** month and year, with the following resulting columns:
* Year (4 digits)
* Month
* Average open, high, low, and close
* Total volume
* Finally, sort this data so it's in proper chronological order

In [6]:
query = """
select 
STRFTIME('%Y', date) as Year,
STRFTIME('%m', date) as Month,
AVG(open) as 'Average open',
AVG(high) as 'Average high',
AVG(low) as 'Average low',
AVG(close) as 'Average close',
SUM(volume) as 'Total volume'
from yum
group by Year, Month
limit 5
"""
pd.read_sql(query, conn)

,Year,Month,Average open,Average high,Average low,Average close,Total volume
0,2015,01,52.221423,52.736520,51.694464,52.192667,89074400
1,2015,02,54.183662,54.823868,53.723561,54.534791,98621800
2,2015,03,56.845958,57.324685,56.445657,56.818509,108827600
3,2015,04,58.533429,59.140050,58.168498,58.740885,117743300
4,2015,05,65.739757,66.475916,65.144141,65.755572,131485600


In [8]:
conn.close()
conn = sqlite3.connect('ladder.db')

###### 64) Save the results of the previous query as a view named `yum_by_month`.

In [9]:
c = conn.cursor()
## c.execute("DROP VIEW IF EXISTS yum_by_month")

c.execute( """ 
CREATE VIEW yum_by_month AS
select 
    STRFTIME('%Y', date) as Year,
    STRFTIME('%m', date) as Month,
    AVG(open) as 'Average open',
    AVG(high) as 'Average high',
    AVG(low) as 'Average low',
    AVG(close) as 'Average close',
    SUM(volume) as 'Total volume'
from yum
group by Year, Month;
""")
conn.commit()

In [ ]:
##conn.close()

In [15]:
query = """
select*
from yum_by_month
limit 5
"""
pd.read_sql(query, conn)

,Year,Month,Average open,Average high,Average low,Average close,Total volume
0,2015,01,52.221423,52.736520,51.694464,52.192667,89074400
1,2015,02,54.183662,54.823868,53.723561,54.534791,98621800
2,2015,03,56.845958,57.324685,56.445657,56.818509,108827600
3,2015,04,58.533429,59.140050,58.168498,58.740885,117743300
4,2015,05,65.739757,66.475916,65.144141,65.755572,131485600


###### 65) Create a view of `transactions` consisting of only three columns: year, month, and total sales in that month. Call this view `trans_by_month`.

In [16]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS trans_by_month")

c.execute( """ 
CREATE VIEW trans_by_month AS
select 
    STRFTIME('%Y', orderdate) as year,
    STRFTIME('%m', orderdate) as month,
    SUM(quantity) as 'total sales'
from transactions
group by Year, Month;
""")
conn.commit()

In [17]:
query = """
select*
from trans_by_month
limit 5
"""
pd.read_sql(query, conn)

,year,month,total sales
0,1990,01,1281
1,1990,02,1010
2,1990,03,832
3,1990,04,1401
4,1990,05,1485


###### 66) Create a view of `transactions` consisting of only two columns: `employee_id` and the total sales corresponding to that employee. Call this view `trans_by_employee`.

In [19]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS trans_by_employee")

c.execute( """ 
CREATE VIEW trans_by_employee AS
select 
    employee_id,
    sum(quantity) as 'total sales'
from transactions
group by employee_id
""")
conn.commit()

In [20]:
query = """
select*
from trans_by_employee
limit 5
"""
pd.read_sql(query, conn)

,employee_id,total sales
0,0,6349
1,1,7635
2,3,6299
3,9,6811
4,10,7463


###### 67) What's the most common first initial for pets in the `pets` table?
    * _Hint:_ Create a CTE that is simply the lowercased first letter of the pet's name. The solution is a simple `GROUP BY` from this CTE.

In [50]:
query = """
select
lower(SUBSTR(name, 1,1)) as pet_first_letter,
count(*) as count
from pets
group by pet_first_letter
order by count desc
limit 1
"""
pd.read_sql(query, conn)

,pet_first_letter,count
0,p,3


In [53]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS pets_first_letter")

c.execute( """ 
CREATE VIEW pets_first_letter AS
select 
   lower(SUBSTR(name, 1,1)) as pet_first_letter,
count(*) as count
from pets
group by pet_first_letter
order by count desc
""")
conn.commit()

In [54]:
query = """
select*
from pets_first_letter
limit 5
"""
pd.read_sql(query, conn)

,pet_first_letter,count
0,p,3
1,m,2
2,s,1
3,o,1
4,g,1


###### 68) Create taglines for each employee in the `employees` table. As a template, the first row of the result should look like this:

In [46]:
query = """
select
firstname ||' '|| lastname || ' started in ' || STRFTIME('%Y', startdate) || ' and makes ' || printf('$%,.2d', salary) || ' working in ' || job || '.' as tagline
from employees;
"""
pd.read_sql(query, conn)

,tagline
0,Christine Thompson started in 2005 and makes $...
1,"Thomas Peck started in 2011 and makes $112,972..."
2,Christopher Robles started in 2003 and makes $...
3,"Elizabeth Munoz started in 1993 and makes $55,..."
4,"Janice Martin started in 2011 and makes $62,00..."
...,...
95,"Tina Flynn started in 2006 and makes $55,916 w..."
96,"Allen Cross started in 2019 and makes $118,571..."
97,Brittany Maynard started in 2003 and makes $69...
98,"Bradley Romero started in 2009 and makes $45,2..."


In [47]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS taglines")

c.execute( """ 
CREATE VIEW taglines AS
select
firstname ||' '|| lastname || ' started in ' || STRFTIME('%Y', startdate) || ' and makes ' || printf('$%,.2d', salary) || ' working in ' || job || '.' as tagline
from employees;
""")
conn.commit()

In [76]:
pd.set_option('display.max_colwidth', 100)
query = """
select*
from taglines
limit 5
"""
pd.read_sql(query, conn)

,tagline
0,"Christine Thompson started in 2005 and makes $123,696 working in Sales."
1,"Thomas Peck started in 2011 and makes $112,972 working in Sales."
2,"Christopher Robles started in 2003 and makes $78,426 working in IT."
3,"Elizabeth Munoz started in 1993 and makes $55,824 working in Sales."
4,"Janice Martin started in 2011 and makes $62,007 working in IT."


###### 67) What's the most common first initial for pets in the `pets` table?
    * _Hint:_ Create a CTE that is simply the lowercased first letter of the pet's name. The solution is a simple `GROUP BY` from this CTE.

In [55]:
query = """
select*
from pets_first_letter
limit 1
"""
pd.read_sql(query, conn)

,pet_first_letter,count
0,p,3


###### 68) Create taglines for each employee in the `employees` table. As a template, the first row of the result should look like this:
```
Christine Thompson started in 2005 and makes $123,696 working in sales. 
```

In [60]:
query = """
select*
from taglines
limit 5
"""
pd.read_sql(query, conn)

,tagline
0,"Christine Thompson started in 2005 and makes $123,696 working in Sales."
1,"Thomas Peck started in 2011 and makes $112,972 working in Sales."
2,"Christopher Robles started in 2003 and makes $78,426 working in IT."
3,"Elizabeth Munoz started in 1993 and makes $55,824 working in Sales."
4,"Janice Martin started in 2011 and makes $62,007 working in IT."


###### 69) How many of our sales come from companies ending in each of "LLC", "Inc", "Ltd", or "PLC"? In a CTE, create a `company_type` column of values `"LLC"`, `"Inc"`, `"Ltd"`, `"PLC"`, or `"Other"`. Outside the CTE, find the total revenue from these categories, as well as their respective counts.
* _Hint:_ You'll need the `INSTR()` function.

In [77]:
query = """
select
    case when customer like '%LLC' then 'LLC'
         when customer like '%Inc' then 'Inc'
         when customer like '%Ltd' then 'Ltd'
         when customer like '%PLC' then 'PLC'
         else 'Other' 
         end as company_type,
    count(*) as total_counts,
from transactions
group by company_type 
order by total_counts
"""
pd.read_sql(query, conn)

,company_type,total_customers
0,LLC,2137
1,PLC,2513
2,Inc,2562
3,Ltd,3603
4,Other,39266


In [79]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS company_types")

c.execute( """ 
CREATE VIEW company_types AS
select 
    case when customer like '%LLC' then 'LLC'
         when customer like '%Inc' then 'Inc'
         when customer like '%Ltd' then 'Ltd'
         when customer like '%PLC' then 'PLC'
         else 'Other' 
         end as company_type,
    count(*) as total_counts
from transactions
group by company_type 
order by total_counts
""")
conn.commit()

In [80]:
query = """
select*
from company_types
limit 5
"""
pd.read_sql(query, conn)

,company_type,total_customers
0,LLC,2137
1,PLC,2513
2,Inc,2562
3,Ltd,3603
4,Other,39266


In [98]:
query = """
select
    case when customer like '%LLC' then 'LLC'
         when customer like '%Inc' then 'Inc'
         when customer like '%Ltd' then 'Ltd'
         when customer like '%PLC' then 'PLC'
         else 'Other' 
         end as company_type,
    count(*) as total_counts,
    sum(unit_price * quantity) as total_revenue
from transactions
group by company_type 
order by total_counts
"""
pd.read_sql(query, conn)

,company_type,total_counts,total_revenue
0,LLC,2137,292365.50
1,PLC,2513,348489.75
2,Inc,2562,343915.75
3,Ltd,3603,481426.00
4,Other,39266,5386857.25


## Joins

###### 70) Which employee made which sale? Join the `employees` table onto the `transactions` table by `employee_id`. You only need to include the employee's first/last name from `employees`.

In [6]:
query = """
select*
from employees
limit 
"""
pd.read_sql(query, conn)

,ID,firstname,lastname,job,salary,startdate
0,0,Christine,Thompson,Sales,123696,2005-01-20


In [7]:
query = """
select
from transactions
limit 1
"""
pd.read_sql(query, conn)

,order_id,customer,unit_price,quantity,orderdate,employee_id
0,0,Bautista Group,20.5,12,2018-10-27,81


In [78]:
query = """
select
  t.order_id,
   e.firstname || ' ' || e.lastname as name,
   t.employee_id,
   t.quantity as sale
from employees e
join transactions t on e.ID = t.employee_id;
"""
pd.read_sql(query, conn)

,order_id,name,employee_id,sale
0,0,Christopher Carlson,81,12
1,0,Christopher Carlson,81,11
2,0,Christopher Carlson,81,14
3,0,Christopher Carlson,81,11
4,0,Christopher Carlson,81,9
...,...,...,...,...
50076,9999,Robert Logan,46,11
50077,9999,Robert Logan,46,11
50078,9999,Robert Logan,46,10
50079,9999,Robert Logan,46,7


###### 71) What is the name of the employee who made the most in sales? Find this answer by doing a join as in the previous problem. Your resulting query will be difficult for someone else to read.

In [77]:
query = """
select
   t.order_id,
   e.firstname || ' ' || e.lastname as name,
   t.employee_id,
   sum(t.quantity) as sales
from employees e
join transactions t on e.ID = t.employee_id
group by t.employee_id
order by sales desc
limit 1;
"""
pd.read_sql(query, conn)

,order_id,name,employee_id,sales
0,66,Anna Fischer,60,9379


73) Solve the previous problem by joining `employees` onto a CTE.

In [41]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS employee_most_sales")

c.execute( """ 
CREATE VIEW employee_most_sales AS
select 
   e.firstname || ' ' || e.lastname as name,
   t.employee_id,
   sum(t.quantity) as sales
from employees e
join transactions t on e.ID = t.employee_id
group by t.employee_id
order by sales desc;
""")
conn.commit()

In [42]:
query = """
select*
from employee_most_sales
limit 5;
"""
pd.read_sql(query, conn)

,name,employee_id,sales
0,Anna Fischer,60,9379
1,Brittany Maynard,97,8765
2,Miguel Williams,73,8236
3,Lisa Morgan,20,8170
4,Shannon Bailey,33,8161


###### 74) Next, the company will try to give bonuses based on performance. Show all employees who've made more in sales than 1.5 times their salary. (You may use whatever technique you'd like to do the join: view, CTE, or even a subquery!)

In [50]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS employee_most_sales")

c.execute( """ 
CREATE VIEW employee_most_sales AS
select
   e.salary,
   e.firstname || ' ' || e.lastname as name,
   t.employee_id,
   sum(t.quantity * t.unit_price) as most_sales
from employees e
join transactions t on e.ID = t.employee_id
group by t.employee_id
order by most_sales desc;
""")
conn.commit()

In [55]:
query = """
select
name,
employee_id,
most_sales/salary as times_salary
from employee_most_sales
order by times_salary desc
limit 5;
"""
pd.read_sql(query, conn)

,name,employee_id,times_salary
0,Miguel Williams,73,3.515103
1,Michael West,65,3.361917
2,Julian Martinez,55,3.245491
3,Candice Wright,85,3.227061
4,Samantha Nichols,26,2.932131


###### 75) Do we have potentially erroneous rows? Find all transactions which occurred _before_ the employee was even hired! (Make sure each transaction only occupies one row).

In [59]:
query = """
select
   t.order_id,
   e.firstname || ' ' || e.lastname as name,
   t.employee_id,
   t.orderdate
from transactions t
join employees e on t.employee_id = e.ID
where t.orderdate < e.startdate;
"""
pd.read_sql(query, conn)

,order_id,name,employee_id,orderdate
0,1,Christine Thompson,0,2003-12-06
1,1,Christine Thompson,0,2003-12-06
2,1,Christine Thompson,0,2003-12-06
3,1,Christine Thompson,0,2003-12-06
4,1,Christine Thompson,0,2003-12-06
...,...,...,...,...
29778,9997,Shannon Bailey,33,2011-08-29
29779,9997,Shannon Bailey,33,2011-08-29
29780,9997,Shannon Bailey,33,2011-08-29
29781,9997,Shannon Bailey,33,2011-08-29


###### 76) Among all transactions that occurred from 2015 to 2019, create a table that is the monthly revenue of our company versus the total trading volume of Yum! in that month. Format the columns nicely. That is, a sample row of your result might look like this:

In [70]:
query = """
select*
from yum
limit 1;
"""
pd.read_sql(query, conn)

,date,open,high,low,close,volume
0,2015-01-02,52.796551,52.846874,51.761322,52.012939,2283400


In [74]:
query = """
select
    STRFTIME('%Y', date) as year,
    STRFTIME('%m', date) as month,
    printf('$%,.2d', SUM(close * volume)) as company_revenue,
    printf('$%,.2d', SUM(volume)) as yum_trade_volume
from yum
where year between '2015-01-01' and '2019-12-31'
group by month
order by month
limit 5;
"""
pd.read_sql(query, conn)

,year,month,company_revenue,yum_trade_volume
0,2016,01,"$15,727,643,881","$244,005,600"
1,2016,02,"$18,796,773,289","$291,911,600"
2,2016,03,"$17,314,212,326","$248,361,400"
3,2016,04,"$14,903,914,855","$210,441,800"
4,2016,05,"$16,750,742,921","$225,902,200"


###### 77) Repeat the previous problem, but in addition to the total volume, include:
* The lowest price that month (ie, lowest low)
* The highest price that month (ie, highest high)

In [12]:
query = """
select
    STRFTIME('%Y', date) as year,
    STRFTIME('%m', date) as month,
    printf('$%,.2d', AVG(low)) as lowest_low,
    printf('$%,.2d', AVG(high)) as highest_high,
    printf('$%,.2d', SUM(close * volume)) as company_revenue,
    printf('$%,.2d', SUM(volume)) as yum_trade_volume
from yum
where year between '2015-01-01' and '2019-12-31'
group by month
order by month
limit 5;
"""
pd.read_sql(query, conn)

,year,month,lowest_low,highest_high,company_revenue,yum_trade_volume
0,2016,01,$72,$73,"$15,727,643,881","$244,005,600"
1,2016,02,$71,$73,"$18,796,773,289","$291,911,600"
2,2016,03,$74,$75,"$17,314,212,326","$248,361,400"
3,2016,04,$77,$78,"$14,903,914,855","$210,441,800"
4,2016,05,$77,$78,"$16,750,742,921","$225,902,200"
